In [13]:
using Distributed
using DataFrames
using DataFramesMeta
using Lazy: @>

@everywhere using Agents

In [28]:
@everywhere include("Initialisation.jl")

@everywhere num_each_type = 100
@everywhere num_steps = 200
@everywhere replicates = 2000

@everywhere multirun_model = evolutionary_model(;
    num_sitters = num_each_type,
    num_identifiers = num_each_type,
    num_cheaters = num_each_type,
    hatch_utility = 4,
    egg_cost = 2,
    identify_cost = 1,
    eggs_laid = 1,
    p_identify = 1,
    p_mutation = 0.1
)



In [29]:
@everywhere sitters(a) = typeof(a) == Sitter
@everywhere identifiers(a) = typeof(a) == Identifier
@everywhere cheaters(a) = typeof(a) == Cheater
@everywhere adata = [(sitters, count), (identifiers, count), (cheaters, count)]

data, _ = run!(multirun_model, agent_step!, model_step!, num_steps; adata = adata, replicates = replicates, parallel=true);

In [31]:
# How good on average is each strategy
@> begin
    data
    @where(:step .== num_steps)
    @combine(
        sitter = mean(:count_sitters)/(num_each_type*3),
        identifiers = mean(:count_identifiers)/(num_each_type*3),
        cheaters = mean(:count_cheaters)/(num_each_type*3)
    )
end

,sitter,identifiers,cheaters
,Float64,Float64,Float64
1,0.344267,0.374377,0.281357
